In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 데이터 합치기 - 뉴스, 블로그, 리뷰(플레이스토어 + 11번가)

#### [ 뉴스 ]

In [25]:
clova_news_df = pd.read_csv("../../data/naver_news/sktnugu_speaker_news.csv", index_col=0)
clova_news_df['source'] = 'news'
clova_news_df.head()

,title,date,publisher,url,text,source
0,"SK텔레콤, 인공지능 대중화 선언…AI 스피커 '누구(NUGU)' 공개",2016.08.31.,아시아경제,https://news.naver.com/main/read.nhn?mode=LSD&...,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news
1,"“스피커가 말을 한다”…SK텔레콤, 인공지능 서비스 ‘NUGU’ 출시",2016.08.31.,매일경제,https://news.naver.com/main/read.nhn?mode=LSD&...,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news
2,"[특징주]아이리버, SKT 인공지능 스피커 ‘누구’ 공동 설계 '강세'",2016.08.31.,파이낸셜뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news
3,"""내일 날씨 어때"" 물으면 ""우산 챙겨요"" 답한다",2016.08.31.,파이낸셜뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news
4,"""사람과 대화하는 스피커""…SKT, AI 서비스 '누구(NUGU)' 공개",NaN,MTN,http://news.mtn.co.kr/newscenter/news_viewer.m...,NaN,news


<br/><br/>
#### [ 블로그 ]

In [27]:
clova_blog_df = pd.read_csv("../../data/naver_blog/nugu_speaker_blog.csv", encoding='utf-16', index_col=0)
clova_blog_df['source'] = 'blog'
clova_blog_df.head()

,title,publish_datetime,text,url,source
0,"SKT 음성인식 스피커 ""NUGU (누구)"" 출시 (like 아마존에코, Amazo...",2016-08-31 11:02:00,\n안녕하세요. 제이전트(Jgent)입니다.드디어 1년 반여년 이상 준비했던 SKT...,https://blog.naver.com/zoophila?Redirect=Log&l...,blog
1,"로봇 비서?…SKT, 음성인식AI ‘누구’ 공개",2016-08-31 16:09:00,1일부터 서비스…인공지능 대중화 시대 선언\n“비 오는 초가을 날씨에 어울리는 노래...,https://blog.naver.com/duoprism?Redirect=Log&l...,blog
2,[2016.8.31 뉴스클리핑] ▲'핀테크 고속도로' 개통…금융권 공동 오픈플랫폼 ...,2016-08-31 12:26:00,세계 첫 사례…핀테크 기업들 빠른 서비스 개발 가능 핀테크 기업이 새로운 서비스를 ...,http://blog.kissoft.co.kr/220801153255,blog
3,NaN,NaN,NaN,http://blog.daum.net/dandakhan/16619935,blog
4,"“ ‘누구’ 신나는 음악좀 틀어줘” SKT, 생활속 AI 로봇 서비스....",2016-08-31 21:19:00,\nSK텔레콤이 지난달 31일 서울시 중구 을지로 본사에서 인공지능 개인비서 로봇 ...,https://blog.naver.com/chase65?Redirect=Log&lo...,blog


<br/>
#### [ 플레이스토어 리뷰 ]

In [28]:
clova_review_df = pd.read_csv('../../data/review/playstore_review_skt_nugu.csv')
clova_review_df['source'] = 'review'
clova_review_df.head()

,name,ratings,date,helpful vote,comment,source
0,임이응,4,2019년 5월 7일,4,안녕하세요. 누구 캔들을 잘 사용하고 있어요. 그런데 제가 아리아를 불렀을 때 아리...,review
1,당당쥐,1,2019년 4월 8일,26,자꾸 연결이 잘 안되어서 미치겠어요. 처음부터 안됬으면 버리기라도 하지 잘되다가 자...,review
2,777 BANG,4,2019년 4월 23일,5,좋고 편리하긴 한데 뭔가 기본적인 것들이 빠져 있음. 업데이트 후 예약 설정해놓은걸...,review
3,king sun Da,1,2019년 5월 9일,0,"아니 어제까지 잘되던데 검색을 하면 연결 기기가 없다고 합니까??? 공유기 초기화,...",review
4,KANGMAN,3,2019년 5월 3일,1,티맵 쓰면서 정말 잘 활용하고 있어요 그런데 QM3의 T2C에서 티맵 실행하고 아리...,review


<br/>
#### [ 11번가 리뷰 ]

In [29]:
_11st_clova_review_df = pd.read_csv('../../data/review/review_data_nugu.csv', index_col=0)
_11st_clova_review_df['source'] = 'review'
_11st_clova_review_df.head()

,index,nugu,score,source
0,1,"와이파이 연결은 자기가알아서 척척 하네요. 무드등 기능 일단좋고,핸펀 블루투스음악보...",5,review
1,2,배송 빠르구요. 제품 하자없어요. 음향 선명하고 맑습니다. 말귀 잘 알아듣구요. 엄...,4,review
2,3,"스피커 상태나 제품도 온전한 것으로 왔는데, 글쎄 제조일이 17년도여서 실망했어요....",3,review
3,4,"귀도 어둡고, 머리도 나쁘고, 그저 노래만 부를 줄 아는군",2,review
4,5,솔직한 사용기 입니다. 일단 음성인식 블루투스 스피커 + 무드등을하나 산다고 해도 ...,1,review


<br/>
#### [ 데이터 합치기 ]


In [30]:
clova_df = pd.DataFrame({"date": pd.concat([clova_news_df.date, clova_blog_df.publish_datetime, 
                                            clova_review_df.date, pd.Series([None] * len(_11st_clova_review_df))]),
                         "text":pd.concat([clova_news_df.text, clova_blog_df.text, 
                                            clova_review_df.comment, _11st_clova_review_df.nugu]),
                         'source':pd.concat([clova_news_df.source, clova_blog_df.source, 
                                             clova_review_df.source, _11st_clova_review_df.source]),
                         'url':pd.concat([clova_news_df.url, clova_blog_df.url, 
                                          pd.Series([None] * len(clova_review_df)), pd.Series([None] * len(_11st_clova_review_df))])
                        })

In [31]:
clova_df = clova_df[clova_df.text.notnull()].copy()
clova_df.reset_index(inplace=True, drop=True)
clova_df

,date,text,source,url
0,2016.08.31.,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2016.08.31.,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2016.08.31.,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2016.08.31.,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2016.08.31.,[머니투데이 이정혁 기자] [사용자와 대화한 만큼 데이터 쌓여 스스로 진화…AI 스...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
5,2016.08.31.,음성인식 전용 스피커 출시…IoT 연결 가전제품도 제어연내 음식 주문 서비스 가능…...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
6,2016.08.31.,[이데일리 김현아 기자]음성인식 기반 인공지능 서비스 ‘누구’(NUGU) 출시 관련...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
7,2016.08.31.,ㆍ인공지능 서비스 ‘NUGU’ 공개ㆍ음성인식 기반 대화하듯 사용ㆍ음악 재생·일정 안...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
8,2016.08.31.,[현장 분석] 탈통신 서비스 '누구'와 두비두'가 던진 화두는 개방과 글로벌 [오마...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
9,2019.06.03.,기업들 '프리미엄 전쟁'[ 김재후 기자 ] SK텔레콤은 디스플레이를 장착한 인공지...,news,https://news.naver.com/main/read.nhn?mode=LSD&...


<br/><br/>
### 텍스트 전처리

In [32]:
def clear_content(temp):
    #want_remove = ['flash 오류를 우회하기 위한 함수 추가', 'function _flash_removeCallback', 
    #               '\n', '\.\.', '[\w]+ 기자', '\xa0', '\r', '\t']
    content = re.sub('[a-zA-Z0-9]+[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', ' ', temp) # 이메일주소 없애기
    content = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w_\.-]*)*\/?', ' ', content) # URL 없애기
    #content = re.sub('|'.join(want_remove), ' ', content)
    content = re.sub('\n|\r|\xa0', ' ', content)
    ㅠcontent = re.sub('[\(]', ' ', content) # 커머스(전자상거래)와' -> '커머스전자상거래와' 이렇게 변하던것을 -> '커머스 전자상거래'
    content = re.sub('~', ' ', content)
    #content = re.sub('[^\.\?\!\^^\♥\♡\w\d\s]','', content) # 불필요한 기호 없애기
    content = re.sub('\d', '', content)
    content = re.sub('[^\.\?\!\^^\♥\♡\w\s]','', content) # 불필요한 기호 없애기
    while '  ' in content: content = content.replace('  ', ' ') # 빈칸 없애주기 
    content = content.strip()
    return content

<br/><br/>
### 모델을 이용해서 네이버 블로그 데이터(中 clova) sentence별 긍정/부정 나누기

In [33]:
clova_df["preprocessed_text"] = clova_df['text'].map(clear_content) 

In [34]:
clova_df.head()

,date,text,source,url,preprocessed_text
0,2016.08.31.,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대 ...
1,2016.08.31.,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...
2,2016.08.31.,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다. 일 오후 시...
3,2016.08.31.,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천 ...
4,2016.08.31.,[머니투데이 이정혁 기자] [사용자와 대화한 만큼 데이터 쌓여 스스로 진화…AI 스...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부터...


In [35]:
del_idx = []
for num, i in enumerate(clova_df['preprocessed_text']):
    if i == '' or i == None or i == ' ':
        del_idx.append(num)
print(del_idx)

[829, 983, 2862, 3020, 3145, 3279, 6138, 6140]


In [36]:
clova_df.loc[clova_df.index[del_idx]]

,date,text,source,url,preprocessed_text
829,2016-10-04 01:13:00,\n\n,blog,https://blog.naver.com/ituwu?Redirect=Log&logN...,
983,2016-12-20 16:59:00,\n\n,blog,https://kappamann.blog.me/220890616238,
2862,2018-08-05 18:09:00,https://blog.naver.com/csjrlove/221327546634,blog,https://blog.naver.com/krbsh?Redirect=Log&logN...,
3020,2018-08-29 17:51:00,http://m.momsdiary.co.kr/w/event/com_event/skt...,blog,https://blog.naver.com/sarahc0c0?Redirect=Log&...,
3145,2018-09-20 16:37:00,\n \n\n,blog,https://blog.naver.com/poka022?Redirect=Log&lo...,
3279,2018-10-26 15:26:00,\n\n,blog,https://blog.naver.com/spine3000?Redirect=Log&...,
6138,2018년 10월 31일,😊,review,None,
6140,2018년 4월 5일,👍,review,None,


In [37]:
final_clova_df = clova_df.drop(clova_df.index[del_idx]).copy()

In [38]:
# 빈 데이터들을 다 삭제하고 난 뒤, 인덱스를 다시 0부터 순서대로 부여해 줌
final_clova_df.reset_index(drop=True, inplace=True)

In [39]:
final_clova_df.head()

,date,text,source,url,preprocessed_text
0,2016.08.31.,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대 ...
1,2016.08.31.,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...
2,2016.08.31.,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다. 일 오후 시...
3,2016.08.31.,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천 ...
4,2016.08.31.,[머니투데이 이정혁 기자] [사용자와 대화한 만큼 데이터 쌓여 스스로 진화…AI 스...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부터...


- 각 문헌을 sentence별로 나누기

In [40]:
import nltk.tokenize
final_clova_df['sentence_text'] = final_clova_df['preprocessed_text'].map(nltk.tokenize.sent_tokenize)

In [41]:
final_clova_df.head()

,date,text,source,url,preprocessed_text,sentence_text
0,2016.08.31.,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대 ...,[아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대...
1,2016.08.31.,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,"[사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다., 이 스피커는 손..."
2,2016.08.31.,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다. 일 오후 시...,"[SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다., 일 오후..."
3,2016.08.31.,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천 ...,[SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천...
4,2016.08.31.,[머니투데이 이정혁 기자] [사용자와 대화한 만큼 데이터 쌓여 스스로 진화…AI 스...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부터...,[머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부...


- 나눈 sentence들을 각각 형태소 분석하기

In [19]:
# 미등록단어 지정하여 Komoran 형태소 분석 
from konlpy.tag import Komoran

#dicpath = './unregistered_dic_blog_test.txt'
#komoran = Komoran(userdic= dicpath)
dicpath = './unregistered.txt'
komoran = Komoran(userdic=dicpath)

In [42]:
final_clova_df['pos_sentence_text'] = final_clova_df['sentence_text'].map(lambda l: [komoran.pos(sentence) for sentence in l])

In [43]:
#[" ".join([wors[0] for word in sent]) for sent in final_df['pos_sentence_text']]
def pos_by_sentence(l):
    result = []
    for sent in l:
        result.append(" ".join([word[0] for word in sent]))
    return result

final_clova_df['pos_sentence_text'] = final_clova_df['pos_sentence_text'].map(pos_by_sentence)

In [44]:
final_clova_df.head()

,date,text,source,url,preprocessed_text,sentence_text,pos_sentence_text
0,2016.08.31.,[아시아경제 박소연 기자]SK텔레콤이 인공지능(AI) 스피커를 선보이며 'AI 대중...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대 ...,[아시아경제 박소연 기자SK텔레콤이 인공지능AI 스피커를 선보이며 AI 대중화 시대...,[아시아 경제 박소연 기자 SK텔레콤 이 인공지능 AI 스피커 를 선보이 며 AI ...
1,2016.08.31.,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다. 이 스피커는 손을 ...,"[사용자의 명령을 수행하고 묻는 말에 대답하는 스피커가 등장했다., 이 스피커는 손...",[사용자 의 명령 을 수행 하 고 묻 는 말 에 대답 하 는 스피커 가 등장 하 았...
2,2016.08.31.,SK텔레콤이 인공지능(AI) 스피커를 선보였다는 소식에 아이리버가 강세다.\n \n...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다. 일 오후 시...,"[SK텔레콤이 인공지능AI 스피커를 선보였다는 소식에 아이리버가 강세다., 일 오후...",[SK텔레콤 이 인공지능 AI 스피커 를 선보이 었 다는 소식 에 아이리버 가 강세...
3,2016.08.31.,"SKT, 음성인식 기반 AI개인비서 서비스 '누구' 선보여가전제품 제어는 물론 음악...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천 ...,[SKT 음성인식 기반 AI개인비서 서비스 누구 선보여가전제품 제어는 물론 음악추천...,[SKT 음성 인식 기반 AI 개인 비서 서비스 누구 선보이 어 가전제품 제어 는 ...
4,2016.08.31.,[머니투데이 이정혁 기자] [사용자와 대화한 만큼 데이터 쌓여 스스로 진화…AI 스...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부터...,[머니투데이 이정혁 기자 사용자와 대화한 만큼 데이터 쌓여 스스로 진화AI 스피커부...,[머니투데이 이정혁 기자 사용자 와 대화 하 ㄴ 만큼 데이터 쌓이 어 스스로 진화 ...


#### sentence별 긍정(1) / 부정(-1)으로 나누기

In [45]:
def label_by_sentence(doc):
    loaded_vectorizer = pickle.load(open('./pretrained_vectorizer.pickle', 'rb'))
    loaded_model = pickle.load(open('./pretrained_model.pickle', 'rb'))
    
    tfidf_features = loaded_vectorizer.transform(doc)
    pred_labels = loaded_model.predict(tfidf_features)
    return pred_labels

final_clova_df['label_sentence_text'] = final_clova_df['pos_sentence_text'].map(label_by_sentence)

In [46]:
final_clova_df.to_csv("누구new.csv", encoding='utf-8')